In [ ]:
import sqlite3
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import FinanceDataReader as fdr
from datetime import date, timedelta, datetime

In [ ]:
end_date_dt = date.today()
start_date_dt = end_date_dt - timedelta(days=60)
end_date = end_date_dt.strftime("%Y-%m-%d")
start_date = start_date_dt.strftime("%Y-%m-%d")

start_date, end_date

In [ ]:
# query financial statement data
conn = sqlite3.connect('../../data/data_v3.1.db')
cur = conn.cursor()

if end_date_dt.month > 5:
    sql = "select * from fs where year=" + str(int(end_date[:4]) - 1) # 작년도의 재무상태표, 손익계산서 데이터를 가져온다. 
else:
    sql = "select * from fs where year=" + str(int(end_date[:4]) - 2) # 재작년도의 재무상태표, 손익계산서 데이터를 가져온다. 
    
cur.execute(sql)
rows = cur.fetchall()
print("Querried rows:", len(rows))

# insert querried data into dataframe
df_company = pd.DataFrame(columns=['날짜', '기업코드', '종목코드', '회사명', '유동자산', '유동부채', '자산총계'])

companies = []
for row in rows:
    company = {}
    company['날짜'] = end_date
    company['기업코드'] = row[1] # corp code
    company['종목코드'] = row[2] # stock code
    company['회사명'] = row[3] # corp name
    company[row[4]] = row[5] # amount
    company['IFRS'] = row[6] # IFRS
    company['CFS'] = row[7] # CFS
    
    companies.append(company)
df_company = df_company.append(companies)

In [ ]:
aggregation_functions = {'종목코드': 'first', '회사명': 'first', '유동자산': 'sum', '유동부채': 'sum', '자산총계': 'sum', '날짜': 'first'}

In [ ]:
# 연결 재무제표 기업
df_cfs = df_company[df_company.CFS == 1]
df_cfs = df_cfs.groupby(['기업코드', 'IFRS', 'CFS']).aggregate(aggregation_functions).reset_index()
df_cfs = df_cfs[df_cfs['자산총계'] > 0]
len(df_cfs)

In [ ]:
# 개별 재무제표 기업
df_fs = df_company[df_company.CFS == 0]
df_fs = df_fs.groupby(['기업코드', 'IFRS', 'CFS']).aggregate(aggregation_functions).reset_index()
df_fs = df_fs[df_fs['자산총계'] > 0]
len(df_fs)

In [ ]:
# CFS가 있는 기업은 FS에서 삭제
cfs_stock_codes = df_cfs['종목코드'].tolist()
df_fs['CFS_EXISTS'] = df_fs.apply(lambda x: x['종목코드'] in cfs_stock_codes, axis=1)
df_fs = df_fs[df_fs['CFS_EXISTS'] == False]
df_fs = df_fs.drop(columns=['CFS_EXISTS'])

# 합체
df_company = pd.concat([df_cfs, df_fs]).reset_index().drop(columns=['index'])
print("Number of companies:", len(df_company))

In [ ]:
df_company['유동비율'] = df_company['유동자산'] / df_company['유동부채']

In [ ]:
df_company

In [ ]:
def get_price_and_stocks(stock_code):
    try:
        url = "https://finance.naver.com/item/main.nhn?code=" + stock_code
        response = requests.get(url)
        html = BeautifulSoup(response.text, 'html.parser')
        
        # 관리종목 여부 확인
        description = html.find("div", {"class": "description"})
        spans = description.find_all("span")
        for span in spans:
            if span.text == '관리종목':
                print("관리종목")
                return 0, 0

        # 주식 수
        aside = html.find("div", {"id": "aside"})
        table = aside.find("div", {"class": "first"})
        trs = table.find_all("tr")

        num_stocks = trs[2].find("td").text
        num_stocks = int(num_stocks.replace(",",""))

        # 가격
        content = html.find("div", {"id": "content"})
        div_today = content.find("div", {"class": "today"})
        price = div_today.find("span").text
        price = int(price.replace(",",""))

        return price, num_stocks
    except:
        return 0, 0

In [ ]:
# 과거 60일 가격 정보로 모멘텀 데이터 채우기
df_company['주가'] = np.NaN
df_company['주식수'] = np.NaN
df_company['시가총액'] = np.NaN
df_company['start_price'] = np.NaN
df_company['end_price'] = np.NaN
df_company['yield'] = np.NaN

for i, row in enumerate(df_company.iterrows()):
    company = row[1] # company = (corp code, ifrs, cfs, stock code, corp name, ...)
    stock_code = company[3]
    
    # 시가총액
    price, num_stocks = get_price_and_stocks(stock_code)
    market_cap = price * num_stocks
    
    df_company.at[row[0], '주가'] = price
    df_company.at[row[0], '주식수'] = num_stocks
    df_company.at[row[0], '시가총액'] = market_cap
    
    if price == 0: # 네이버 금융에 가격정보가 없거나 현재 관리종목인 경우
        print(i, "no price NAVER:", company[4], stock_code)
        continue
    else:
        df_price = fdr.DataReader(stock_code, start_date, end_date).reset_index()
        
        if len(df_price) == 0:
            print(i, "no price FDR:", company[4])
            continue
            
        # 마지막 가격 날짜 확인
        df_price_end_date = df_price.iloc[len(df_price) - 1].Date
        df_price_end_date = datetime.strptime(str(df_price_end_date)[:10], '%Y-%m-%d').date()

        if (end_date_dt - df_price_end_date).days >= 7: # 최근 일주일 내 거래되지 않았으면 제거
            print(i, "최근 거래 없음:", company[4], df_price_end_date)
            continue

        start_price = df_price.iloc[0].Open
        end_price = df_price.iloc[len(df_price) - 1].Close
        stock_yield = end_price / start_price 
        
        df_company.at[row[0], 'start_price'] = start_price
        df_company.at[row[0], 'end_price'] = end_price
        df_company.at[row[0], 'yield'] = stock_yield
        
        print(i, company[4], stock_code, market_cap, stock_yield)

In [ ]:
# # test
df_price = fdr.DataReader('084870', start_date, end_date).reset_index()
df_price